# Modello AI per Quick Draw doodle recognition
Per installare le dipendenze necessarie definite in requirements.txt
```bash
pip install -r requirements.txt
```
oppure
```bash
pip install torch torch torchvision onnx requests
```

In [ ]:
! pip install torch torchvision torchaudio onnx requests

In [ ]:
from torch import nn
from torch.optim import Adam
from torch import cuda
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from typing import List, Optional
import urllib.request
from pathlib import Path
import requests
import math
import numpy as np

In [ ]:
#Imposto il device su GPU se disponibile
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
#Funzione per ottenere tutti i nomi delle classi di quickdraw (non usata)
def get_all_quickdraw_class_names():
    url = "https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt"
    r = requests.get(url)
    classes: List = [x.replace(' ', '_') for x in r.text.splitlines()]
    return classes

#Lista limitata di classi di quickdraw
def get_quickdraw_class_names():
    classes = ['Eiffel Tower','Tent','Airplane','Ambulance','Apple','Asparagus','Banana','Baseball','Basketball','Birthday Cake','T-shirt','Triangle','Elephant','Guitar','Rainbow','Lighthouse','Television','Snowman','Penguin','Coffee Cup']
    classes = sorted([x.replace(' ', '_').lower() for x in classes])
    return classes

In [ ]:
# Download del dataset di quickdraw da google storage bucket
def download_quickdraw_dataset(root="./dataset", limit: Optional[int] = None, class_names: List[str]=None, select_all:bool =False):
    if class_names is None:
        class_names = get_quickdraw_class_names()
    if select_all:
        class_names = get_all_quickdraw_class_names()

    root = Path(root)
    root.mkdir(exist_ok=True, parents=True)
    url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

    print("Downloading Quickdraw Dataset...")
    for class_name in class_names[:limit]:
        fpath = root / f"{class_name}.npy"
        if not fpath.exists():
            urllib.request.urlretrieve(f"{url}{class_name.replace('_', '%20')}.npy", fpath)

In [ ]:
def load_quickdraw_data(root="./dataset", max_items_per_class=5000):
    all_files = Path(root).glob('*.npy')

    x = np.empty([0, 784], dtype=np.uint8) #Data
    y = np.empty([0], dtype=np.longlong) #Labels
    class_names = [] #Nomi delle classi

    print(f"Loading {max_items_per_class} examples for each class from the Quickdraw Dataset...")
    
    for idx, file in enumerate(sorted(all_files)):
        #Carica i dati da ogni file
        data = np.load(file, mmap_mode='r')
        #Tronca i dati a max_items_per_class
        data = data[0: max_items_per_class, :]
        #Aggiunge i dati e le labels
        labels = np.full(data.shape[0], idx)
        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)
        class_names.append(file.stem)

    return x, y, class_names

In [ ]:

#Dataset di quickdraw per pytorch ereditando da torch.utils.data.Dataset
class QuickDrawDataset(torch.utils.data.Dataset):
    def __init__(self, root, max_items_per_class=5000, class_limit=None, transform=None):
        super().__init__()
        self.root = root
        self.max_items_per_class = max_items_per_class
        self.class_limit = class_limit
        self.transform = transform
        download_quickdraw_dataset(self.root, self.class_limit)
        self.X, self.Y, self.classes = load_quickdraw_data(self.root, self.max_items_per_class)

    def __getitem__(self, idx):
        x = (self.X[idx] / 255.).astype(np.float32).reshape(1, 28, 28)
        y = self.Y[idx]
        if self.transform:
            data = self.transform(x)

        return torch.from_numpy(x), y.item()

    def __len__(self):
        return len(self.X)

    def collate_fn(self, batch):
        x = torch.stack([item[0] for item in batch])
        y = torch.LongTensor([item[1] for item in batch])
        return {'pixel_values': x, 'labels': y}
    
    def split(self, pct=0.1):
        num_classes = len(self.classes)
        indices = torch.randperm(len(self)).tolist()
        n_val = math.floor(len(indices) * pct)
        train_ds = torch.utils.data.Subset(self, indices[:-n_val])
        noTransform = self
        noTransform.transform = None
        val_ds = torch.utils.data.Subset(noTransform, indices[-n_val:])
        return train_ds, val_ds

In [ ]:

data_dir = './dataset'
max_examples_per_class = 20000
train_val_split_pct = .1

data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(28),  # assuming input images are of size 224x224
])

ds = QuickDrawDataset(data_dir, max_examples_per_class, transform=data_transforms)
num_classes = len(ds.classes)
print(f"Number of classes: {num_classes}")
#Dataset di training e di validazione
train_ds, val_ds = ds.split(train_val_split_pct)

## Define the Model

In [ ]:
class Model(nn.Module):
    #Constuttore
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(2304, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        return self.model(x)

model = Model().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
# Funzione di valutare il modello con dati che non ha mai visto
def evaluate(model):
    correct = 0
    total = 0
    for data, target in DataLoader(val_ds, batch_size=64):
        data = data.to(device)
        target = target.to(device)
        pred = model(data)
        correct += (pred.argmax(1) == target).type(torch.float).sum().item()
        total += target.size(0)
    print(f'Accuracy: {(correct/total)*100}%')

In [ ]:
# Fuzione per esportare il modello in formato ONNX
def toONNX(model, filename):
    #                        (batch, channel, width and height)
    dummy_input = torch.randn(1, 1, 28, 28).to(device)
    torch.onnx.export(model, dummy_input, filename, verbose=True)

## Train

In [ ]:
if __name__ == '__main__':
    for epoch in range(15):
        for idx_batch, (data,target) in enumerate(DataLoader(train_ds, batch_size=64)):
            data = data.to(device)
            target = target.to(device)

            # Stampa cosa viene classificato
            # print(get_quickdraw_class_names()[target[1]])
            # display(transforms.functional.to_pil_image(data[1]))

            y_hat = model(data).to(device)
            loss = loss_fn(y_hat, target)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            if idx_batch % 3000 == 0:
                print(f'Epoch: {epoch}, Loss: {loss.item()}')

    model.eval()

    torch.save(model.state_dict(), 'model.pth')

    print('Done training')

    evaluate(model)

    toONNX(model, 'model.onnx')